In [1]:
# Call the relevant python packages
import numpy as np
import glob, os
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import scipy.constants as const

from astropy import units as u
from astropy.io import ascii
from astropy.table import Table

from sedfitter import filter_output, write_parameters, write_parameter_ranges
from sedfitter.extinction import Extinction
from sedfitter import fit, plot

import seaborn as sns
sns.set_context("paper")

%matplotlib inline

In [2]:
# Read in the SED formatted file containing the high probability YSOs
SED_input_file = 'High_Prob_YSO_SED.txt'
data = ascii.read(SED_input_file)

# Print the number of potential YSOs found in the table
YSO_num=len(data)
print("The number of YSO candidates in the table is",YSO_num)

The number of YSO candidates in the table is 105


In [3]:
# Read in extinction law
extinction = Extinction.from_file('Input_SED/kmh94.par', columns=[0, 3], wav_unit=u.micron, chi_unit=u.cm**2 / u.g)

# Define filters and apertures
filters = ['2J', '2H', '2K', 'I1', 'I2', 'I3', 'I4', 'M1']
apertures = [2., 2., 2., 1.2, 1.2, 1.2, 1.2, 6.] * u.arcsec

models_list = ['s---s-i','s---smi','s-p-hmi','s-p-smi','s-pbhmi','s-pbsmi','s-u-hmi','s-u-smi','s-ubhmi',
               's-ubsmi','sp--h-i','sp--hmi','sp--s-i','sp--smi','spu-hmi','spu-smi']

'''
# Run the fitting for each model set (18 total)
# Creates output files which can be saved and plotted later
for i in range(len(models_list)):
    model_dir = 'models_YSO_2017/'+models_list[i]
    SED_output_file = '2017_outputs/'+models_list[i]+'.fitinfo'
    fit(SED_input_file, filters, apertures, model_dir,
        SED_output_file,
        extinction_law=extinction,
        distance_range=[450., 530.] * u.kpc,     # Distance range here is for NGC 6822
        av_range=[0., 40.])                      # A_V range here is for NGC 6822
'''

"\n# Run the fitting for each model set (18 total)\n# Creates output files which can be saved and plotted later\nfor i in range(len(models_list)):\n    model_dir = 'models_YSO_2017/'+models_list[i]\n    SED_output_file = '2017_outputs/'+models_list[i]+'.fitinfo'\n    fit(SED_input_file, filters, apertures, model_dir,\n        SED_output_file,\n        extinction_law=extinction,\n        distance_range=[450., 530.] * u.kpc,     # Distance range here is for NGC 6822\n        av_range=[0., 40.])                      # A_V range here is for NGC 6822\n"

In [12]:
for i in range(len(models_list)):
    fitinfo_input_file = '2017_outputs/'+models_list[i]+'.fitinfo'
    #ranges_output_file = '2017_outputs/'+models_list[i]+'_ranges.txt'
    #write_parameter_ranges(fitinfo_input_file,ranges_output_file,select_format=('F',1.))
    params_output_file = '2017_outputs/'+models_list[i]+'_params.txt'
    write_parameters(fitinfo_input_file,params_output_file,select_format=('F',1.))

In [4]:
# Define solar radius, Stefan-Boltzmann constant and temperature of the Sun
sigma = const.Stefan_Boltzmann
R_sun = 6.957e+08 * u.m
T_sun = 5778 * u.Kelvin
        
# Define a list of parameters to find MADs for
param_list = ['chi2','av','star.radius','star.temperature']

for m in range(len(models_list)):
    params_import_file = '2017_outputs/'+models_list[m]+'_params.txt'
    p_data = ascii.read(params_import_file, data_start=1)
    ranges_import_file = '2017_outputs/'+models_list[m]+'_ranges.txt'
    pr_data = ascii.read(ranges_import_file, data_start=1)

    # Create columns for luminosity in both files and calculate in units of L☉
    p_data['luminosity']=0
    p_data['luminosity']=(((p_data['star.radius'])**2)*((p_data['star.temperature'])**4)/T_sun**4)
    pr_data['ltot_best']=0
    pr_data['ltot_best']=(((pr_data['star.radius_best'])**2)*((pr_data['star.temperature_best'])**4)/T_sun**4)

    # Print the total number of fits in the parameter file
    print("\n",models_list[m])
    num_fits = len(p_data['source_name'])
    print("The number of fits in this file is",num_fits)
    # Print the total number of sources in the ranges file
    num_sources = len(pr_data['source_name'])
    print("The number of sources in this file is",num_sources)

    # Build array of n_fits corresponding to each source.
    # This writes a '1' to a new column to flag if that row starts a new source
    source_num = 0
    p_data['new_source?']=0
    arr_new_source = np.array([])
    for i in range(num_fits):
        new_source = p_data['source_name'][i] - source_num
        if new_source == 1:
            arr_new_source = np.append(arr_new_source,np.array([i]))
            p_data['new_source?'][i]=1
        source_num = p_data['source_name'][i]
    
    # Find MAD for each parameter above
    # Set up empty columns for each MAD in ranges file
    p_data['L_MAD']=0
    pr_data['L_MAD']=0
    for i in range(len(param_list)):
        param_mad_name = param_list[i]+'_MAD'
        p_data[param_mad_name]=0
        pr_data[param_mad_name]=0
    
    # For each source in the table find the MADs
    idx_new_source = np.where(p_data['new_source?']==1)

    # Cycle through each fit for each source to get luminosity MAD
    counter = 0
    for i in range(num_sources):
        counter = counter + 1
        L_best = pr_data['ltot_best'][i]
        n_fits_for_source = pr_data['n_fits'][i]
        arr_MAD = np.array([])
        if n_fits_for_source > 1:
            for j in range(n_fits_for_source-1):
                dev = np.array([abs(L_best - p_data['luminosity'][counter])])
                arr_MAD = np.append(arr_MAD, dev)
                counter = counter + 1
            MAD_value = np.median(arr_MAD)
        else:
            MAD_value = 0
        pr_data['L_MAD'][i] = MAD_value
        
    # Run this again for all other non-user-calculated parameters
    # Cycle through all parameters listed above
    for n in range(len(param_list)):
        counter = 0
        for i in range(num_sources):
            counter = counter + 1
            param_best_name = param_list[n]+'_best'
            param_best = pr_data[param_best_name][i]
            n_fits_for_source = pr_data['n_fits'][i]
            arr_MAD = np.array([])
            if n_fits_for_source > 1:
                for j in range(n_fits_for_source-1):
                    dev = np.array([abs(param_best - p_data[param_list[n]][counter])])
                    arr_MAD = np.append(arr_MAD, dev)
                    counter = counter + 1
                MAD_value = np.median(arr_MAD)
            else:
                MAD_value = 0
            param_mad_name = param_list[n]+'_MAD'
            pr_data[param_mad_name][i] = MAD_value
         
    pr_out_name = '2017_outputs/'+models_list[m]+'_ranges_analysed.csv'
    p_out_name = '2017_outputs/'+models_list[m]+'_params_analysed.csv'
    #ascii.write(pr_data,pr_out_name,format='csv')
    #ascii.write(p_data,p_out_name,format='csv')


 s---s-i
The number of fits in this file is 2223
The number of sources in this file is 105

 s---smi
The number of fits in this file is 8077
The number of sources in this file is 105

 s-p-hmi
The number of fits in this file is 434
The number of sources in this file is 105

 s-p-smi
The number of fits in this file is 1107
The number of sources in this file is 105

 s-pbhmi
The number of fits in this file is 2881
The number of sources in this file is 105

 s-pbsmi
The number of fits in this file is 2298
The number of sources in this file is 105

 s-u-hmi
The number of fits in this file is 4560
The number of sources in this file is 105

 s-u-smi
The number of fits in this file is 15305
The number of sources in this file is 105

 s-ubhmi
The number of fits in this file is 4391
The number of sources in this file is 105

 s-ubsmi
The number of fits in this file is 3027
The number of sources in this file is 105

 sp--h-i
The number of fits in this file is 1261
The number of sources in this 

In [13]:
# Create new table for all final parameters
table = Table()
table['source.name'] = pr_data['source_name']
table['best_model'] = ''
table['best_n_fits'] = 0

# For each source in the csv
for i in range(num_sources):
    max_fits = 0
    # For each model set
    for j in range(len(models_list)):
        ranges_import_file = '2017_outputs/'+models_list[m]+'_ranges.txt'
        pr_data = ascii.read(ranges_import_file, data_start=1)
        number_fits = pr_data['n_fits'][i]
        if number_fits > max_fits:
            max_fits = number_fits
            best_model = models_list[j]
            print(i,j)
    table['best_n_fits'][i] = max_fits
    table['best_model'][i] = best_model

table.show_in_notebook()

0 0


C:\Users\Mikey\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: StringTruncateWarning: truncated right side string(s) longer than 1 character(s) during assignment


1 0


C:\Users\Mikey\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: StringTruncateWarning: truncated right side string(s) longer than 1 character(s) during assignment


2 0


...

C:\Users\Mikey\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: StringTruncateWarning: truncated right side string(s) longer than 1 character(s) during assignment


30 0


C:\Users\Mikey\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: StringTruncateWarning: truncated right side string(s) longer than 1 character(s) during assignment


31 0


KeyboardInterrupt: 